In [ ]:
# Imported Dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

# Extract CSV Files to DataFrames

In [ ]:
# Exctract first football dataset and create a dataframe
nfl_file1 = "Resources/NFL_Salaries1.csv"
nfl_df1 = pd.read_csv(nfl_file1)
nfl_df1.head()

In [ ]:
# Exctract second football dataset and create a dataframe
nfl_file2 = "Resources/NFL_Salaries2.csv"
nfl_df2 = pd.read_csv(nfl_file2)
nfl_df2.head()

In [ ]:
# Exctract first basketball dataset and create a dataframe
nba_file1 = "Resources/salaries_1985to2018.csv"
nba_df1 = pd.read_csv(nba_file1)
nba_df1.head()

In [ ]:
# Exctract second basketball dataset and create a dataframe
nba_file2 = "Resources/players.csv"
nba_df2 = pd.read_csv(nba_file2)
nba_df2.head()

In [ ]:
# Exctract first baseball dataset and create a dataframe
mlb_file1 = "Resources/Hitters_Adjusted_Salary.csv"
mlb_df1 = pd.read_csv(mlb_file1)
mlb_df1.head()

In [ ]:
# Exctract second baseball dataset and create a dataframe
mlb_file2 = "Resources/Ptichers_Adjusted_Salary.csv"
mlb_df2 = pd.read_csv(mlb_file2)
mlb_df2.head()

# Transform Football DataFrame

In [ ]:
# Append DataFrames to create a single DataFrame for the football data
all_nfl = nfl_df1.append(nfl_df2)

# Drop columns with excessive null values Roster Bonus, Option Bonus, Restruc. Bonus, and Misc.
all_nfl.drop(columns=["Row ID", "Roster Bonus", "Option Bonus", "Restruc. Bonus", "Misc."], axis=1, inplace=True)

# Rename columns to create a uniform dataframe
all_nfl = all_nfl.rename(columns={"Year": "Years",
                                  "Pos.": "Pos",
                                  "Cap Type": "Cap_Type",
                                  "Base Salary": "Base_Salary",
                                  "Signing Bonus": "Signing_Bonus",
                                  "Workout Bonus": "Workout_Bonus",
                                  "Dead Cap": "Dead_Cap",
                                  "Cap Hit": "Cap_Hit",
                                  "Cap %": "Cap_Percent"})


# Further cleaned the data by dropping rows containing null values indicated by "-" in the dataset
all_nfl = all_nfl[all_nfl['Base_Salary'] != "-"]
all_nfl = all_nfl[all_nfl['Signing_Bonus'] != "-"]
all_nfl = all_nfl[all_nfl['Workout_Bonus'] != "-"]
all_nfl = all_nfl[all_nfl['Dead_Cap'] != "-"]
all_nfl = all_nfl[all_nfl['Cap_Hit'] != "-"]

# Clean the data by setting the index and dropping null values                             
all_nfl.dropna(inplace=True) 
all_nfl.reset_index(drop =True, inplace=True)

# Display first five rows of the dataframe
all_nfl

# Transform Basketball DataFrame

In [ ]:
# Create a filtered dataframe from the first basketball dataframe using specific columns
basketball_cols = ["player_id", "salary", "season_start", "team"]
basketball_df1 = nba_df1[basketball_cols].copy()

# Rename the second basketball dataframe and renamed the "_id" column
basketball_df2 = nba_df2.rename(columns = {"_id": "player_id"})

# Merge the two dataframes using the "player_id" column
all_nba = pd.merge(basketball_df1, basketball_df2, how="left", on = ["player_id", "player_id"])

# Rename columns to create a uniform dataframe
all_nba = all_nba.rename(columns={"career_FG%": "career_FG_Percent",
                                  "career_FG3%": "career_FG3_Percent",
                                  "career_FT%": "career_FT_Percent",
                                  "career_eFG%": "career_eFG_Percent"
                                  })

# Clean the data by dropping null values
all_nba.dropna(inplace=True)

# Display the first five rows of the dataframe
all_nba.head()

# Transform Baseball DataFrame

In [ ]:
# Create a dataframe by merging the two baseball dataframes using the "playerID" column
all_mlb = pd.merge(mlb_df1, mlb_df2, on = ["playerID", "playerID"], suffixes=("_hitter", "_pitcher"))

# Rename columns to create a uniform dataframe
all_mlb = all_mlb.rename(columns={"Unnamed: 0_hitter": "Unnamed_0_hitter",
                                  "ADJ Salary_hitter": "ADJ_Salary_hitter",
                                  "Unnamed: 0_pitcher": "Unnamed_0_pitcher",
                                  "ADJ Salary_pitcher": "ADJ_Salary_pitcher"})

# Check the dataframe for null values
all_mlb.isnull().sum().sum()
# The above line of code returns 0 confirming there are no null values

# Display the first five rows of the dataframe
all_mlb.head()


# Create Database Connection

In [ ]:
connection_string = "postgres:postgres@localhost:5432/Sports_Salaries"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

# Load DataFrames into Database

In [ ]:
all_nfl.to_sql(name='football', con=engine, if_exists='append', index=True)

In [ ]:
all_nba.to_sql(name='basketball', con=engine, if_exists='append', index=True)

In [ ]:
all_mlb.to_sql(name='baseball', con=engine, if_exists='append', index=True)